# 5.後片付け
このセッションで利用したリソースの削除を行います。

In [ ]:
import boto3

In [ ]:
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')

In [ ]:
print(role)
print(region)
print(account_id)

## StepFunctions
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/stepfunctions.html#SFN.Client.delete_state_machine

In [ ]:
stepfunctions = boto3.client('stepfunctions')

In [ ]:
stepfunctions.list_state_machines(
    maxResults=123,
)

In [ ]:
stepfunctions.delete_state_machine(
    stateMachineArn=f'arn:aws:states:{region}:{account_id}:stateMachine:exp-preprocess-train-predict-evaluate'
)

## Lambda

https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/lambda.html#Lambda.Client.delete_function

In [ ]:
boto3lambda = boto3.client('lambda')

In [ ]:
response = boto3lambda.list_functions(
    #MasterRegion='string',
    #FunctionVersion='ALL',
    #Marker='string',
    MaxItems=123
)

In [ ]:
boto3lambda.delete_function(
    FunctionName='lambda_pipeline_dispatcher',
    #Qualifier='string'
)

## CodeCommit
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/codecommit.html#CodeCommit.Client.delete_repository

In [ ]:
codecommit = boto3.client('codecommit')

In [ ]:
codecommit.list_repositories(
    #nextToken='string',
    #sortBy='repositoryName'|'lastModifiedDate',
    #order='ascending'|'descending'
)

In [ ]:
codecommit.delete_repository(
    repositoryName='demo-exp-project1'
)

## S3バケット
https://www.kabegiwablog.com/entry/2019/11/10/142007

SageMaker ノートブックインスタンスの実行ロールにS3Fullが必要

In [ ]:
project_name = 'project1'
Bucket=f'demo-exp-pipeline-{project_name}',


s3 = boto3.resource('s3')
bucket = s3.Bucket(f'demo-exp-pipeline-{project_name}')

In [ ]:
Bucket

In [ ]:
### オブジェクト削除
s3_objects = list(bucket.objects.all())
for s3_object in s3_objects:
    s3_object.delete()

In [ ]:
### オブジェクトバージョン削除
s3_object_versions = list(bucket.object_versions.all())
for s3_object_version in s3_object_versions:
    s3_object_version.delete()

In [ ]:
### バケット削除
bucket.delete()

## IAMロール
lambdaとSFnの
* Roleの削除
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/iam.html#IAM.Client.delete_role

* Policyの削除
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/iam.html#IAM.Client.delete_policy

In [ ]:
iam_client = boto3.client('iam')

### Lambdaに対して

https://www.kabegiwablog.com/entry/2019/10/12/073928

In [ ]:
### ロールの削除
role_name = 'demo-AWSLambda-ExperimentPipelineDispatcher-Role'

session = boto3.Session()
iam = session.resource('iam')

role = iam.Role(role_name)

policies = list(role.attached_policies.all())
for policy in policies:
    policy.detach_role(RoleName=role_name)

instance_profiles = list(role.instance_profiles.all())
for instance_profile in instance_profiles:
    instance_profile.remove_role(RoleName=role_name)
    instance_profile.delete()

role.delete()

In [ ]:
### policy削除
response = iam_client.delete_policy(
    PolicyArn=f'arn:aws:iam::{account_id}:policy/demo-AWSLambda-ExperimentPipelineDispatcher-Policy'
)

### SFnに対して

In [ ]:
### ロールの削除
role_name = 'demo-StepFunctions-ExperimentPipeline-Role'

session = boto3.Session()
iam = session.resource('iam')

role = iam.Role(role_name)

policies = list(role.attached_policies.all())
for policy in policies:
    policy.detach_role(RoleName=role_name)

instance_profiles = list(role.instance_profiles.all())
for instance_profile in instance_profiles:
    instance_profile.remove_role(RoleName=role_name)
    instance_profile.delete()

role.delete()

In [ ]:
### policy削除
response = iam_client.delete_policy(
    PolicyArn=f'arn:aws:iam::{account_id}:policy/demo-StepFunctions-ExperimentPipeline-Policy'
)

## Jupyterノートブック実行環境は、それぞれ適切な方法で終了してください。
* SageMakerノートブックインスタンスの場合は、インスタンスをstopしてください。
* SageMaker Studioの場合は、アプリケーションをstopしてください。